In [15]:
from numpy import *

In [16]:
U , sigma , VT = linalg.svd([[1,1],[7,7]])

In [17]:
U

array([[-0.14142136, -0.98994949],
       [-0.98994949,  0.14142136]])

In [18]:
sigma

array([10.,  0.])

In [19]:
VT

array([[-0.70710678, -0.70710678],
       [-0.70710678,  0.70710678]])

In [56]:
def loadExData():
    return [[1,1,1,0,0],
             [2,2,2,0,0], 
             [1,1,1,0,0],
           [5,5,5,0,0],
           [1,1,0,2,2],
           [0,0,0,3,3],
           [0,0,0,1,1]]

def loadExData_():
    return [[4,4,0,2,2],
             [4,0,0,3,3], 
             [4,0,0,1,1],
           [1,1,1,2,0],
           [2,2,2,0,0],
           [1,1,1,0,0],
           [5,5,5,0,0]]

def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [21]:
Data = loadExData()

U , Sigma , VT = linalg.svd(Data)

In [22]:
Sigma

array([9.72140007e+00, 5.29397912e+00, 6.84226362e-01, 1.50962387e-15,
       1.15387192e-31])

In [23]:
Sig3 = mat([[Sigma[0] , 0 , 0],
           [0,Sigma[1] , 0],
           [0,0,Sigma[2]]])

In [24]:
U[:,:3]*Sig3*VT[:3 , :]

matrix([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         -2.84366098e-16, -2.94015497e-16],
        [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
          4.47489534e-16,  4.28190736e-16],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          3.09573758e-16,  2.99924358e-16],
        [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00,
         -1.47703573e-16, -1.95842150e-16],
        [ 1.00000000e+00,  1.00000000e+00, -5.70229711e-16,
          2.00000000e+00,  2.00000000e+00],
        [-7.49390630e-17,  9.96896569e-16, -1.34350906e-15,
          3.00000000e+00,  3.00000000e+00],
        [-8.18314124e-17,  2.75447132e-16, -3.13743829e-16,
          1.00000000e+00,  1.00000000e+00]])

In [25]:
from numpy import linalg as la

In [26]:
def euclidSim(inA , inB):
    return 1.0/(1.0+la.norm(inA - inB)) #欧几里得距离来衡量距离

def pearsSim(inA , inB):
    if len(inA)<3:
        return 1.0
    
    return 0.5+0.5*corrcoef(inA , inB , rowvar=0)[0][1]

def cosSim(inA , inB):
    num = float(inA.T*inB)
    
    demon = la.norm(inA)*la.norm(inB)
    
    return 0.5+0.5*(num/demon)

In [31]:
myMat = mat(loadExData())

In [32]:
euclidSim(myMat[:,0] , myMat[:,4])

0.13367660240019172

In [33]:
euclidSim(myMat[:,0] , myMat[:,0])

1.0

In [34]:
cosSim(myMat[:,0] , myMat[:,0])

0.9999999999999999

In [50]:
def standEst(dataMat , user , simMeas , item):
    n = shape(dataMat)[1]
    
    simTotal = 0.0
    ratSimTotal = 0.0
    
    for j in range(n):
        userRating = dataMat[user , j]
        
        if userRating == 0:
            continue
        
        overLap = nonzero(logical_and(dataMat[: , item].A>0,
                                      dataMat[: , j].A))[0]
        
        if len(overLap) == 0:
            similarity = 0
        else:
            print(dataMat[overLap , item] , dataMat[overLap , j] , 'hhh')
            similarity = simMeas(dataMat[overLap , item] , dataMat[overLap , j])
        
        #print
        
        simTotal += similarity
        
        ratSimTotal += similarity*userRating
    
    if simTotal == 0:
        return 0
    else:
        return ratSimTotal/simTotal


def recommend(dataMat , user , N=3 , simMeas = cosSim , estMethod=standEst):
    unratedItems = nonzero(dataMat[user , :].A == 0)[1]
    
    if len(unratedItems) == 0:
        return 'everything realted'
    
    itemScores = []
    
    for item in unratedItems:
        estimatedScore = estMethod(dataMat , user , simMeas , item)
        print('ttt')
        itemScores.append((item , estimatedScore))
    
    return sorted(itemScores , key=lambda jj: jj[1] , reverse=True)[:N]

In [40]:
myMat = mat(loadExData_())

In [41]:
myMat

matrix([[4, 4, 0, 2, 2],
        [4, 0, 0, 3, 3],
        [4, 0, 0, 1, 1],
        [1, 1, 1, 2, 0],
        [2, 2, 2, 0, 0],
        [1, 1, 1, 0, 0],
        [5, 5, 5, 0, 0]])

In [42]:
recommend(myMat , 2)

[(2, 2.5), (1, 2.0243290220056256)]

In [43]:
recommend(myMat , 2 , simMeas=euclidSim)

[(2, 3.0), (1, 2.8266504712098603)]

In [51]:
recommend(myMat , 2 , simMeas=pearsSim)

(matrix([[4],
        [1],
        [2],
        [1],
        [5]]), matrix([[4],
        [1],
        [2],
        [1],
        [5]]), 'hhh')
(matrix([[4],
        [1]]), matrix([[2],
        [2]]), 'hhh')
(matrix([[4]]), matrix([[2]]), 'hhh')
ttt
(matrix([[1],
        [2],
        [1],
        [5]]), matrix([[1],
        [2],
        [1],
        [5]]), 'hhh')
(matrix([[1]]), matrix([[2]]), 'hhh')
ttt


[(2, 2.5), (1, 2.0)]

In [57]:
U , Sigma , VT = la.svd(mat(loadExData2()))

In [58]:
Sigma

array([15.77075346, 11.40670395, 11.03044558,  4.84639758,  3.09292055,
        2.58097379,  1.00413543,  0.72817072,  0.43800353,  0.22082113,
        0.07367823])

In [59]:
Sig2 = Sigma**2

In [60]:
sum(Sig2)

541.9999999999995

In [61]:
sum(Sig2[:3])

500.50028912757926

In [63]:
def svdEst(dataMat , user , simMeas , item):
    n = shape(dataMat)[1]
    simTotal = 0.0
    ratSimTotal = 0.0
    
    U , Sigma , VT = la.svd(dataMat)
    
    Sig4 = mat(eye(4)*Sigma[:4])
    xformedItems = dataMat.T * U[:,:4]*Sig4.I
    
    for j in range(n):
        userRating = dataMat[user , j]
        
        if userRating == 0 or j==item:
            continue
        
        similarity = simMeas(xformedItems[item , :].T , xformedItems[j,:].T)
        
        print('the %d and %d similarity is :%f' % (item , j , similarity))
        
        simTotal += similarity
        
        ratSimTotal += similarity*userRating
        
    if simTotal == 0:
        return 0
    else:
        return ratSimTotal/simTotal

In [65]:
recommend(myMat  , 2 , estMethod=svdEst)

the 1 and 0 similarity is :0.498142
the 1 and 3 similarity is :0.498131
the 1 and 4 similarity is :0.509974
ttt
the 2 and 0 similarity is :0.552670
the 2 and 3 similarity is :0.552976
the 2 and 4 similarity is :0.217301
ttt


[(2, 2.253270755977715), (1, 1.9921514636756923)]

In [66]:
recommend(myMat , 2 , estMethod=svdEst , simMeas=pearsSim)

the 1 and 0 similarity is :0.626075
the 1 and 3 similarity is :0.672793
the 1 and 4 similarity is :0.614375
ttt
the 2 and 0 similarity is :0.429334
the 2 and 3 similarity is :0.387057
the 2 and 4 similarity is :0.043539
ttt


[(2, 2.497798373616037), (1, 1.9816972841840976)]

In [79]:
def printMat(inMat , thresh=0.8):
    for i in range(32):
        for j in range(32):
            if float(inMat[i,j])>thresh:
                print 1 , 
            else:
                print 0 ,
        print ''
    

def imgCompress(numSV=3 , thresh=0.8):
    myl = []
    
    for line in open('../MLiA_SourceCode/Ch14/0_5.txt').readlines():
        newRow = []
        
        for i in range(32):
            newRow.append(int(line[i]))
        
        myl.append(newRow)
    
    myMat = mat(myl)
    
    print('******origin matrix*********')
    
    printMat(myMat , thresh)
    U , Sigma , VT = la.svd(myMat)
    
    SigRecon = mat(zeros((numSV , numSV)))
    
    for k in range(numSV):
        SigRecon[k,k] = Sigma[k]
    
    reconMat = U[:,:numSV]*SigRecon*VT[:numSV , :]
    
    print('reconstructed matrix using % singular values********' % numSV)
    
    printMat(reconMat , thresh)

In [80]:
imgCompress(2)

******origin matrix*********
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0